# BatteryDataTool Jupyter Notebook 예제

UI_Button_Function_Flow.md를 기반으로 구현한 Jupyter Notebook 예제입니다.

## 📚 주요 기능

1. **기본 유틸리티 함수** (9개)
2. **파일 시스템 함수** (3개)
3. **PNE 데이터 처리**
4. **그래프 생성** (6종류)
5. **데이터 탐색**
6. **종합 파이프라인**
7. **실제 데이터 로딩**
8. **통합 Cycle 분석**
9. **충방전 프로파일 분석**
10. **HPPC/DCIR 분석**

## 🎯 데이터 모드

이 노트북은 **2가지 모드**로 실행됩니다:

### A. 실제 데이터 모드 (권장)
- `BatteryDataTool.py`가 있고 `Rawdata/` 폴더에 데이터가 있을 때
- 실제 배터리 데이터 분석

### B. 샘플 데이터 모드
- 실제 데이터가 없거나 Import 실패할 때
- 시뮬레이션 데이터로 모든 기능 테스트

---
## Section 1: 환경 설정 및 Import

In [ ]:
# 기본 라이브러리
import os
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.signal import savgol_filter
from scipy.signal import find_peaks
from dataclasses import dataclass
from typing import Optional, Tuple, Dict, List

# 한글 폰트 설정
plt.rcParams["font.family"] = "Malgun Gothic"  # Windows
plt.rcParams["axes.unicode_minus"] = False

# 경로 설정
BASE_PATH = r"c:\Users\Ryu\Python_project\data\battery251027"
RAWDATA_PATH = os.path.join(BASE_PATH, "Rawdata")

print(f"✅ 기본 라이브러리 Import 완료")
print(f"📁 Base Path: {BASE_PATH}")
print(f"📁 Rawdata Path: {RAWDATA_PATH}")

In [ ]:
# BatteryDataTool.py 함수 Import 시도
USE_REAL_DATA = False

try:
    from BatteryDataTool import (
        progress, extract_text_in_brackets, name_capacity,
        check_cycler,
        pne_min_cap, pne_search_cycle,
        pne_cycle_data, toyo_cycle_data,
        pne_chg_Profile_data, pne_dchg_Profile_data,
        toyo_chg_Profile_data, toyo_dchg_Profile_data,
        pne_dcir_Profile_data,
        graph_cycle, graph_profile, graph_soc_dcir, graph_dcir
    )
    USE_REAL_DATA = True
    print("✅ BatteryDataTool.py 함수 Import 성공")
    print("📊 실제 데이터 모드 활성화")
except ImportError as e:
    print("⚠️ BatteryDataTool.py 함수 Import 실패")
    print(f"   오류: {e}")
    print("📝 샘플 데이터 모드로 전환합니다.")
    USE_REAL_DATA = False

---
## Section 2: 기본 유틸리티 함수 (9개)

BatteryDataTool.py에서 자주 사용되는 유틸리티 함수들입니다.

In [ ]:
# 1. progress() - 진행률 계산
if USE_REAL_DATA:
    # 실제 함수 테스트
    # progress(count1, max1, count2, max2, count3, max3)
    prog = progress(5, 10, 50, 100, 1000, 1000)
    print(f"진행률 테스트 (실제 함수): {prog:.2f}%")
else:
    # 샘플 구현
    def progress(count1, max1, count2, max2, count3, max3):
        file_prog = (count1 / max1) * 25 if max1 > 0 else 0
        cycle_prog = (count2 / max2) * 50 if max2 > 0 else 0
        cap_prog = (count3 / max3) * 25 if max3 > 0 else 0
        return file_prog + cycle_prog + cap_prog
    
    prog = progress(5, 10, 50, 100, 1000, 1000)
    print(f"진행률 테스트 (샘플): {prog:.2f}%")

print(f"\n예제: 파일 5/10, 사이클 50/100, 용량 1000/1000 → {prog:.2f}%")

In [ ]:
# 2. extract_text_in_brackets() - 대괄호 내 텍스트 추출
if USE_REAL_DATA:
    # 실제 함수 테스트
    text = "Test_4500mAh_23C"
    result = extract_text_in_brackets(text)
    print(f"대괄호 추출 (실제): '{text}' → {result}")
else:
    # 샘플 구현
    def extract_text_in_brackets(text):
        import re
        match = re.search(r'\[([^\]]+)\]', text)
        return match.group(1) if match else None
    
    text = "Test_[4500mAh]_23C"
    result = extract_text_in_brackets(text)
    print(f"대괄호 추출 (샘플): '{text}' → {result}")

# 추가 테스트
test_cases = ["[4500mAh]", "Test_[23C]_Data", "NobracketsHere"]
for test in test_cases:
    result = extract_text_in_brackets(test)
    print(f"  '{test}' → {result}")

In [ ]:
# 3. name_capacity() - 폴더명에서 용량 추출
if USE_REAL_DATA:
    # 실제 함수 테스트
    folder_name = "Test_4500mAh_23C"
    capacity = name_capacity(folder_name)
    print(f"용량 추출 (실제): '{folder_name}' → {capacity} mAh")
else:
    # 샘플 구현
    def name_capacity(folder_name):
        import re
        match = re.search(r'(\d+)\s*mAh', folder_name, re.IGNORECASE)
        return int(match.group(1)) if match else 0
    
    folder_name = "Test_4500mAh_23C"
    capacity = name_capacity(folder_name)
    print(f"용량 추출 (샘플): '{folder_name}' → {capacity} mAh")

# 추가 테스트
test_folders = ["4500mAh_Test", "Battery_3000 mAh", "5200mah_cycle", "NoCapacityHere"]
for folder in test_folders:
    cap = name_capacity(folder)
    print(f"  '{folder}' → {cap} mAh")

In [ ]:
# 4-9. 나머지 유틸리티 함수들
print("\n=== 추가 유틸리티 함수 ===\n")

# 샘플 구현 (USE_REAL_DATA=False인 경우)
if not USE_REAL_DATA:
    def name_cycler(folder_name):
        """폴더명에서 충방전기 타입 추출"""
        if 'PNE' in folder_name.upper():
            return 'PNE'
        elif 'TOYO' in folder_name.upper():
            return 'TOYO'
        return 'UNKNOWN'
    
    def name_crate(folder_name):
        """폴더명에서 C-rate 추출"""
        import re
        match = re.search(r'(\d+\.?\d*)C', folder_name)
        return float(match.group(1)) if match else 0.2
    
    def name_temp(folder_name):
        """폴더명에서 온도 추출"""
        import re
        match = re.search(r'([-+]?\d+)\s*C(?!\d)', folder_name)
        return int(match.group(1)) if match else 25
    
    def name_cutoff_v(folder_name):
        """폴더명에서 cutoff 전압 추출"""
        import re
        match = re.search(r'(\d+\.?\d*)V', folder_name)
        return float(match.group(1)) if match else 2.5
    
    def name_rest_t(folder_name):
        """폴더명에서 rest 시간 추출"""
        import re
        match = re.search(r'rest(\d+)', folder_name, re.IGNORECASE)
        return int(match.group(1)) if match else 10
    
    def split_string_num(text):
        """문자열과 숫자 분리"""
        import re
        match = re.match(r'([a-zA-Z]+)(\d+)', text)
        if match:
            return match.group(1), int(match.group(2))
        return text, 0

# 테스트
test_folder = "Test_4500mAh_PNE_1C_45C_3.0V_rest30"
print(f"테스트 폴더: {test_folder}\n")

if USE_REAL_DATA:
    from BatteryDataTool import name_cycler, name_crate, name_temp, name_cutoff_v, name_rest_t, split_string_num

print(f"4. name_cycler(): {name_cycler(test_folder)}")
print(f"5. name_crate(): {name_crate(test_folder)} C")
print(f"6. name_temp(): {name_temp(test_folder)} °C")
print(f"7. name_cutoff_v(): {name_cutoff_v(test_folder)} V")
print(f"8. name_rest_t(): {name_rest_t(test_folder)} min")
print(f"9. split_string_num('Ch001'): {split_string_num('Ch001')}")

---
## Section 3: 파일 시스템 함수 (3개)

In [ ]:
# 1. remove_end_comma() - CSV 파일 끝 쉼표 제거
if not USE_REAL_DATA:
    def remove_end_comma(file_path):
        """CSV 파일 끝의 쉼표 제거"""
        try:
            with open(file_path, 'r', encoding='utf-8') as f:
                lines = f.readlines()
            
            cleaned_lines = [line.rstrip(',\n') + '\n' for line in lines]
            
            with open(file_path, 'w', encoding='utf-8') as f:
                f.writelines(cleaned_lines)
            return True
        except Exception as e:
            print(f"오류: {e}")
            return False

print("remove_end_comma() 함수 준비 완료")
print("  사용법: remove_end_comma('path/to/file.csv')")
print("  기능: CSV 파일의 각 줄 끝에 있는 불필요한 쉼표 제거")

In [ ]:
# 2. check_cycler() - 충방전기 타입 확인 (PNE vs TOYO)
if not USE_REAL_DATA:
    def check_cycler(folder_path):
        """폴더 구조로 충방전기 타입 판별"""
        pattern_path = os.path.join(folder_path, "Pattern")
        restore_path = os.path.join(folder_path, "Restore")
        
        # PNE: Pattern/ 및 Restore/ 폴더 존재
        if os.path.exists(pattern_path) and os.path.exists(restore_path):
            return True  # PNE
        
        # TOYO: 채널 폴더 (001, 002 등) 존재
        if os.path.exists(folder_path):
            subdirs = [d for d in os.listdir(folder_path) if os.path.isdir(os.path.join(folder_path, d))]
            if any(d.isdigit() for d in subdirs):
                return False  # TOYO
        
        return None  # 알 수 없음

# 테스트
print("check_cycler() 함수 준비 완료")
print("  반환값: True (PNE), False (TOYO), None (알 수 없음)")
print("  PNE 식별: Pattern/ 및 Restore/ 폴더 존재")
print("  TOYO 식별: 채널 폴더 (001, 002, ...) 존재")

if os.path.exists(RAWDATA_PATH):
    print(f"\n  Rawdata 폴더 확인: {RAWDATA_PATH}")
    folders = [f for f in os.listdir(RAWDATA_PATH) if os.path.isdir(os.path.join(RAWDATA_PATH, f))]
    if folders:
        test_folder = os.path.join(RAWDATA_PATH, folders[0])
        result = check_cycler(test_folder)
        cycler_type = "PNE" if result else "TOYO" if result is False else "알 수 없음"
        print(f"  첫 번째 폴더: {folders[0]} → {cycler_type}")
else:
    print(f"\n  ⚠️ Rawdata 폴더가 없습니다: {RAWDATA_PATH}")

In [ ]:
# 3. Rawdata 폴더 탐색 함수
def explore_rawdata(rawdata_path):
    """Rawdata 폴더 구조 탐색 및 분석"""
    if not os.path.exists(rawdata_path):
        print(f"❌ 경로가 존재하지 않습니다: {rawdata_path}")
        return []
    
    folders = [f for f in os.listdir(rawdata_path) if os.path.isdir(os.path.join(rawdata_path, f))]
    
    if not folders:
        print(f"⚠️ 폴더가 비어있습니다: {rawdata_path}")
        return []
    
    print(f"📁 총 {len(folders)}개 폴더 발견\n")
    
    results = []
    for folder in folders:
        folder_path = os.path.join(rawdata_path, folder)
        capacity = name_capacity(folder)
        is_pne = check_cycler(folder_path)
        cycler_type = "PNE" if is_pne else "TOYO" if is_pne is False else "UNKNOWN"
        
        results.append({
            'folder': folder,
            'path': folder_path,
            'capacity': capacity,
            'cycler': cycler_type
        })
        
        print(f"  [{cycler_type:7s}] {folder:40s} ({capacity} mAh)")
    
    return results

# 실행
print("=== Rawdata 폴더 탐색 ===\n")
rawdata_folders = explore_rawdata(RAWDATA_PATH)

---
## Section 4: PNE 데이터 처리 함수

In [ ]:
# pne_min_cap() - PNE 데이터에서 최소 용량 산정
if not USE_REAL_DATA:
    def pne_min_cap(folder_path, ini_crate=0.2):
        """첫 사이클 전류값으로 용량 추정 (샘플)"""
        # 샘플: 폴더명에서 추출하거나 기본값
        capacity = name_capacity(os.path.basename(folder_path))
        if capacity > 0:
            return capacity
        # 기본값
        return 4500

# 테스트
if rawdata_folders and rawdata_folders[0]['cycler'] == 'PNE':
    test_path = rawdata_folders[0]['path']
    capacity = pne_min_cap(test_path, ini_crate=0.2)
    print(f"pne_min_cap() 테스트: {capacity} mAh")
else:
    print("pne_min_cap() 함수 준비 완료 (테스트 데이터 없음)")

In [ ]:
# pne_search_cycle() - PNE 데이터에서 특정 사이클 검색
if not USE_REAL_DATA:
    def pne_search_cycle(folder_path, target_cycle):
        """특정 사이클 번호의 데이터 파일 찾기 (샘플)"""
        restore_path = os.path.join(folder_path, "Restore")
        if not os.path.exists(restore_path):
            return None
        
        # SaveData####.csv 형식 파일 찾기
        files = [f for f in os.listdir(restore_path) if f.startswith("SaveData") and f.endswith(".csv")]
        
        # 사이클 번호 매칭 (간단한 버전)
        for f in files:
            # 실제로는 파일을 읽어서 사이클 번호 확인해야 함
            return os.path.join(restore_path, f)
        
        return None

print("pne_search_cycle() 함수 준비 완료")
print("  기능: 특정 사이클 번호의 상세 데이터 파일 찾기")

---
## Section 5: 실제 데이터 로딩 Helper 함수

In [ ]:
# Settings 클래스 정의
@dataclass
class CycleAnalysisSettings:
    """사이클 분석 설정"""
    capacity: int = 0  # 0이면 자동 산정
    ini_crate: float = 0.2
    chkir: bool = True
    cutoff: float = 0.0
    cycle_range_x: int = 0  # 0이면 자동
    capacity_y_min: float = 0.70
    capacity_y_max: float = 1.05
    dcir_scale: float = 1.5
    save_excel: bool = False

@dataclass
class ProfileSettings:
    """프로파일 분석 설정"""
    capacity: int = 0
    cycle_nums: str = "2 10 50"  # 공백으로 구분
    smooth_window: int = 51
    peak_prominence: float = 0.5
    save_excel: bool = False

@dataclass
class HPPCSettings:
    """HPPC/DCIR 분석 설정"""
    capacity: int = 0
    start_cycle: int = 1
    end_cycle: int = 5
    pulse_duration: float = 10.0  # seconds
    save_excel: bool = False

print("✅ Settings 클래스 정의 완료")

In [ ]:
def load_cycle_data_real(folder_path: str, capacity: int = 0, ini_crate: float = 0.2, chkir: bool = True) -> Tuple[int, pd.DataFrame, str]:
    """
    실제 데이터 로딩 (PNE/TOYO 자동 감지) with 자동 샘플 데이터 fallback
    
    Returns:
        capacity (int): 용량 (mAh)
        cycle_df (pd.DataFrame): 사이클 데이터
        cycler_type (str): "PNE", "TOYO", 또는 "SAMPLE"
    """
    
    # 실제 데이터 모드일 때만 로딩 시도
    if USE_REAL_DATA:
        try:
            # 충방전기 타입 확인
            is_pne = check_cycler(folder_path)
            
            if is_pne:
                # PNE 데이터 로딩
                if capacity == 0:
                    capacity = pne_min_cap(folder_path, ini_crate)
                
                # pne_cycle_data(raw_file_path, mincapacity, ini_crate, chkir, chkir2, mkdcir)
                result = pne_cycle_data(folder_path, capacity, ini_crate, chkir, False, True)
                if result is not None and len(result) > 0:
                    cycle_df = result
                    return capacity, cycle_df, "PNE"
            
            elif is_pne is False:
                # TOYO 데이터 로딩
                if capacity == 0:
                    capacity = name_capacity(os.path.basename(folder_path))
                    if capacity == 0:
                        capacity = 4500  # 기본값
                
                # toyo_cycle_data(raw_file_path, mincapacity, inirate, chkir)
                result = toyo_cycle_data(folder_path, capacity, ini_crate, chkir)
                if result is not None and len(result) > 0:
                    cycle_df = result
                    return capacity, cycle_df, "TOYO"
        
        except Exception as e:
            print(f"⚠️ 실제 데이터 로딩 실패: {e}")
            print("📝 샘플 데이터로 전환합니다.")
    
    # 샘플 데이터 생성
    print("📝 샘플 데이터를 생성합니다.")
    
    if capacity == 0:
        capacity = name_capacity(os.path.basename(folder_path))
        if capacity == 0:
            capacity = 4500
    
    # 샘플 사이클 데이터 생성 (100 사이클)
    cycles = np.arange(1, 101)
    
    # 방전 용량 (서서히 감소)
    dchg = 1.0 - 0.002 * cycles + np.random.normal(0, 0.01, len(cycles))
    dchg = np.clip(dchg, 0.7, 1.0)
    
    # 충전 용량 (방전보다 약간 높음)
    chg = dchg + 0.02 + np.random.normal(0, 0.01, len(cycles))
    chg = np.clip(chg, 0.7, 1.05)
    
    # 효율
    eff = dchg / chg
    eff2 = chg / dchg
    
    # 온도 (23°C ± 2°C)
    temp = 23 + np.random.normal(0, 2, len(cycles))
    
    # DCIR (서서히 증가)
    dcir = 50 + 0.5 * cycles + np.random.normal(0, 5, len(cycles))
    
    # 전압
    rnd_v = 3.7 - 0.001 * cycles + np.random.normal(0, 0.05, len(cycles))
    avg_v = 3.65 - 0.001 * cycles + np.random.normal(0, 0.05, len(cycles))
    
    cycle_df = pd.DataFrame({
        'Dchg': dchg,
        'Chg': chg,
        'Eff': eff,
        'Eff2': eff2,
        'Temp': temp,
        'dcir': dcir,
        'RndV': rnd_v,
        'AvgV': avg_v
    }, index=cycles)
    
    return capacity, cycle_df, "SAMPLE"

print("✅ load_cycle_data_real() 함수 정의 완료")

In [ ]:
def load_profile_data_real(folder_path: str, capacity: int, cycle_num: int, profile_type: str = 'charge') -> Optional[Dict]:
    """
    프로파일 데이터 로딩 (충전 또는 방전) with 자동 샘플 데이터 fallback
    
    Args:
        folder_path: 데이터 폴더 경로
        capacity: 용량 (mAh)
        cycle_num: 사이클 번호
        profile_type: 'charge' 또는 'discharge'
    
    Returns:
        profile (dict): {'time', 'voltage', 'current', 'capacity', 'dqdv'}
    """
    
    # 실제 데이터 모드일 때만 로딩 시도
    if USE_REAL_DATA:
        try:
            is_pne = check_cycler(folder_path)
            
            if is_pne:
                # PNE 프로파일 데이터
                # 함수 시그니처: (raw_file_path, inicycle, mincapacity, cutoff, inirate, smoothdegree)
                if profile_type == 'charge':
                    result = pne_chg_Profile_data(folder_path, cycle_num, capacity, 0.0, 0.2, 51)
                else:
                    result = pne_dchg_Profile_data(folder_path, cycle_num, capacity, 0.0, 0.2, 51)
                
                if result is not None:
                    return result
            
            elif is_pne is False:
                # TOYO 프로파일 데이터
                # 함수 시그니처: (raw_file_path, inicycle, mincapacity, cutoff, inirate, smoothdegree)
                if profile_type == 'charge':
                    result = toyo_chg_Profile_data(folder_path, cycle_num, capacity, 0.0, 0.2, 51)
                else:
                    result = toyo_dchg_Profile_data(folder_path, cycle_num, capacity, 0.0, 0.2, 51)
                
                if result is not None:
                    return result
        
        except Exception as e:
            print(f"⚠️ 실제 프로파일 데이터 로딩 실패: {e}")
            print("📝 샘플 데이터로 전환합니다.")
    
    # 샘플 프로파일 데이터 생성
    print(f"📝 샘플 {profile_type} 프로파일 생성 (Cycle {cycle_num})")
    
    if profile_type == 'charge':
        # 충전 프로파일 (CC-CV)
        time = np.linspace(0, 2.5, 500)  # 2.5시간
        
        # CC 구간 (0~2h): 전압 상승
        cc_time = time[time <= 2.0]
        cc_voltage = 3.0 + 0.7 * (cc_time / 2.0)
        cc_current = np.ones_like(cc_time) * capacity * 0.5  # 0.5C
        
        # CV 구간 (2~2.5h): 전류 감소
        cv_time = time[time > 2.0]
        cv_voltage = np.ones_like(cv_time) * 4.2
        cv_current = capacity * 0.5 * np.exp(-5 * (cv_time - 2.0))
        
        voltage = np.concatenate([cc_voltage, cv_voltage])
        current = np.concatenate([cc_current, cv_current])
    
    else:
        # 방전 프로파일
        time = np.linspace(0, 2.0, 500)  # 2시간
        voltage = 4.2 - 1.2 * (time / 2.0) - 0.1 * np.sin(10 * time / 2.0)
        current = -np.ones_like(time) * capacity * 0.5  # -0.5C
    
    # 용량 계산 (적분)
    cap_values = np.cumsum(np.abs(current) * np.gradient(time))
    
    # dQ/dV 계산
    dv = np.gradient(voltage)
    dq = np.gradient(cap_values)
    dqdv = np.where(np.abs(dv) > 0.001, dq / dv, 0)
    
    # Smoothing
    if len(dqdv) > 51:
        dqdv = savgol_filter(dqdv, 51, 3)
    
    profile = {
        'time': time,
        'voltage': voltage,
        'current': current,
        'capacity': cap_values,
        'dqdv': dqdv
    }
    
    return profile

print("✅ load_profile_data_real() 함수 정의 완료")

---
## Section 6: 데이터 로딩 테스트

In [ ]:
# 사이클 데이터 로딩 테스트
print("=== 사이클 데이터 로딩 테스트 ===\n")

if rawdata_folders:
    test_folder = rawdata_folders[0]['path']
    print(f"테스트 폴더: {os.path.basename(test_folder)}")
    
    capacity, cycle_df, cycler_type = load_cycle_data_real(
        folder_path=test_folder,
        capacity=0,  # 자동 산정
        ini_crate=0.2,
        chkir=True
    )
    
    print(f"\n✅ 로딩 성공")
    print(f"   충방전기: {cycler_type}")
    print(f"   용량: {capacity} mAh")
    print(f"   사이클 수: {len(cycle_df)}")
    print(f"\n데이터 미리보기:")
    print(cycle_df.head())
    print(f"\n통계:")
    print(cycle_df.describe())

else:
    print("⚠️ 테스트할 폴더가 없습니다. 샘플 데이터를 사용합니다.")
    
    # 샘플 경로로 테스트
    test_folder = os.path.join(RAWDATA_PATH, "Sample_4500mAh_Test")
    capacity, cycle_df, cycler_type = load_cycle_data_real(test_folder, 0, 0.2, True)
    
    print(f"\n✅ 샘플 데이터 생성 완료")
    print(f"   모드: {cycler_type}")
    print(f"   용량: {capacity} mAh")
    print(f"   사이클 수: {len(cycle_df)}")
    print(f"\n데이터 미리보기:")
    print(cycle_df.head())

---
## Section 7: 그래프 생성 함수

In [ ]:
def plot_cycle_analysis(cycle_df: pd.DataFrame, capacity: int, settings: CycleAnalysisSettings):
    """
    사이클 분석 그래프 (6개)
    1. 방전 용량
    2. 충전 용량
    3. 충방 효율
    4. 방충 효율
    5. DCIR
    6. 온도
    """
    
    fig, axes = plt.subplots(3, 2, figsize=(15, 12))
    fig.suptitle(f'사이클 분석 결과 (용량: {capacity} mAh)', fontsize=16, fontweight='bold')
    
    cycles = cycle_df.index.values
    
    # 1. 방전 용량
    axes[0, 0].plot(cycles, cycle_df['Dchg'], 'o-', color='#2E86AB', linewidth=2, markersize=4)
    axes[0, 0].set_xlabel('Cycle', fontsize=12)
    axes[0, 0].set_ylabel('방전 용량 비율', fontsize=12)
    axes[0, 0].set_title('1. 방전 용량', fontsize=12, fontweight='bold')
    axes[0, 0].set_ylim(settings.capacity_y_min, settings.capacity_y_max)
    axes[0, 0].grid(True, alpha=0.3)
    
    # 2. 충전 용량
    axes[0, 1].plot(cycles, cycle_df['Chg'], 'o-', color='#A23B72', linewidth=2, markersize=4)
    axes[0, 1].set_xlabel('Cycle', fontsize=12)
    axes[0, 1].set_ylabel('충전 용량 비율', fontsize=12)
    axes[0, 1].set_title('2. 충전 용량', fontsize=12, fontweight='bold')
    axes[0, 1].set_ylim(settings.capacity_y_min, settings.capacity_y_max)
    axes[0, 1].grid(True, alpha=0.3)
    
    # 3. 충방 효율
    axes[1, 0].plot(cycles, cycle_df['Eff'], 'o-', color='#F18F01', linewidth=2, markersize=4)
    axes[1, 0].set_xlabel('Cycle', fontsize=12)
    axes[1, 0].set_ylabel('효율', fontsize=12)
    axes[1, 0].set_title('3. 충방 효율 (Dchg/Chg)', fontsize=12, fontweight='bold')
    axes[1, 0].set_ylim(0.95, 1.02)
    axes[1, 0].grid(True, alpha=0.3)
    
    # 4. 방충 효율
    axes[1, 1].plot(cycles, cycle_df['Eff2'], 'o-', color='#C73E1D', linewidth=2, markersize=4)
    axes[1, 1].set_xlabel('Cycle', fontsize=12)
    axes[1, 1].set_ylabel('효율', fontsize=12)
    axes[1, 1].set_title('4. 방충 효율 (Chg/Dchg)', fontsize=12, fontweight='bold')
    axes[1, 1].set_ylim(0.98, 1.05)
    axes[1, 1].grid(True, alpha=0.3)
    
    # 5. DCIR
    axes[2, 0].plot(cycles, cycle_df['dcir'], 'o-', color='#6A994E', linewidth=2, markersize=4)
    axes[2, 0].set_xlabel('Cycle', fontsize=12)
    axes[2, 0].set_ylabel('DCIR (mΩ)', fontsize=12)
    axes[2, 0].set_title('5. DCIR 변화', fontsize=12, fontweight='bold')
    axes[2, 0].grid(True, alpha=0.3)
    
    # 6. 온도
    axes[2, 1].plot(cycles, cycle_df['Temp'], 'o-', color='#BC4B51', linewidth=2, markersize=4)
    axes[2, 1].set_xlabel('Cycle', fontsize=12)
    axes[2, 1].set_ylabel('온도 (°C)', fontsize=12)
    axes[2, 1].set_title('6. 온도 변화', fontsize=12, fontweight='bold')
    axes[2, 1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    # 통계 출력
    print("\n=== 사이클 분석 통계 ===\n")
    print(f"총 사이클 수: {len(cycle_df)}")
    print(f"\n방전 용량:")
    print(f"  초기: {cycle_df['Dchg'].iloc[0]:.4f} ({cycle_df['Dchg'].iloc[0]*capacity:.1f} mAh)")
    print(f"  최종: {cycle_df['Dchg'].iloc[-1]:.4f} ({cycle_df['Dchg'].iloc[-1]*capacity:.1f} mAh)")
    print(f"  용량 유지율: {(cycle_df['Dchg'].iloc[-1]/cycle_df['Dchg'].iloc[0])*100:.2f}%")
    print(f"\n평균 충방 효율: {cycle_df['Eff'].mean():.4f}")
    print(f"평균 DCIR: {cycle_df['dcir'].mean():.2f} mΩ")
    print(f"평균 온도: {cycle_df['Temp'].mean():.2f} °C")

print("✅ plot_cycle_analysis() 함수 정의 완료")

In [ ]:
# 사이클 분석 그래프 테스트
settings = CycleAnalysisSettings()
plot_cycle_analysis(cycle_df, capacity, settings)

---
## Section 8: 프로파일 분석

In [ ]:
def plot_profile_analysis(profiles: List[Dict], cycle_nums: List[int], profile_type: str = 'charge'):
    """
    충전/방전 프로파일 분석 그래프
    1. 전압 프로파일
    2. 전류 프로파일
    3. dQ/dV 그래프
    """
    
    fig, axes = plt.subplots(1, 3, figsize=(18, 5))
    title = '충전' if profile_type == 'charge' else '방전'
    fig.suptitle(f'{title} 프로파일 분석', fontsize=16, fontweight='bold')
    
    colors = ['#2E86AB', '#A23B72', '#F18F01', '#C73E1D', '#6A994E']
    
    for i, (profile, cycle_num) in enumerate(zip(profiles, cycle_nums)):
        color = colors[i % len(colors)]
        label = f'Cycle {cycle_num}'
        
        # 1. 전압 프로파일
        axes[0].plot(profile['time'], profile['voltage'], '-', color=color, linewidth=2, label=label)
        axes[0].set_xlabel('시간 (h)', fontsize=12)
        axes[0].set_ylabel('전압 (V)', fontsize=12)
        axes[0].set_title('1. 전압 프로파일', fontsize=12, fontweight='bold')
        axes[0].grid(True, alpha=0.3)
        axes[0].legend()
        
        # 2. 전류 프로파일
        axes[1].plot(profile['time'], profile['current'], '-', color=color, linewidth=2, label=label)
        axes[1].set_xlabel('시간 (h)', fontsize=12)
        axes[1].set_ylabel('전류 (mA)', fontsize=12)
        axes[1].set_title('2. 전류 프로파일', fontsize=12, fontweight='bold')
        axes[1].grid(True, alpha=0.3)
        axes[1].legend()
        
        # 3. dQ/dV
        axes[2].plot(profile['voltage'], profile['dqdv'], '-', color=color, linewidth=2, label=label)
        axes[2].set_xlabel('전압 (V)', fontsize=12)
        axes[2].set_ylabel('dQ/dV', fontsize=12)
        axes[2].set_title('3. dQ/dV 분석', fontsize=12, fontweight='bold')
        axes[2].grid(True, alpha=0.3)
        axes[2].legend()
    
    plt.tight_layout()
    plt.show()
    
    # Peak 검출 (마지막 사이클)
    print(f"\n=== {title} 프로파일 Peak 분석 (Cycle {cycle_nums[-1]}) ===\n")
    last_profile = profiles[-1]
    peaks, properties = find_peaks(last_profile['dqdv'], prominence=0.5, distance=20)
    
    if len(peaks) > 0:
        print(f"검출된 Peak 수: {len(peaks)}")
        for i, peak_idx in enumerate(peaks):
            peak_v = last_profile['voltage'][peak_idx]
            peak_dqdv = last_profile['dqdv'][peak_idx]
            print(f"  Peak {i+1}: V={peak_v:.3f}V, dQ/dV={peak_dqdv:.2f}")
    else:
        print("Peak가 검출되지 않았습니다.")

print("✅ plot_profile_analysis() 함수 정의 완료")

In [ ]:
# 프로파일 분석 테스트
print("=== 충전 프로파일 분석 테스트 ===\n")

profile_settings = ProfileSettings()
cycle_nums = [int(x) for x in profile_settings.cycle_nums.split()]

# 실제 폴더 또는 샘플
if rawdata_folders:
    test_folder = rawdata_folders[0]['path']
else:
    test_folder = os.path.join(RAWDATA_PATH, "Sample_4500mAh_Test")

# 여러 사이클 로딩
profiles = []
for cycle_num in cycle_nums:
    profile = load_profile_data_real(test_folder, capacity, cycle_num, 'charge')
    if profile:
        profiles.append(profile)

if profiles:
    plot_profile_analysis(profiles, cycle_nums[:len(profiles)], 'charge')
else:
    print("⚠️ 프로파일 데이터를 로딩할 수 없습니다.")

---
## Section 9: HPPC/DCIR 분석

In [ ]:
def generate_hppc_sample_data(capacity: int, start_cycle: int, end_cycle: int) -> Dict:
    """
    HPPC 샘플 데이터 생성
    """
    soc_points = np.linspace(100, 10, 10)  # 100% → 10% (10단계)
    cycles = range(start_cycle, end_cycle + 1)
    
    hppc_data = {
        'soc': soc_points,
        'cycles': {}
    }
    
    for cycle in cycles:
        # DCIR: SOC가 낮을수록 증가, 사이클이 진행될수록 증가
        base_dcir = 40 + (cycle - start_cycle) * 5
        dcir = base_dcir + (100 - soc_points) * 0.3 + np.random.normal(0, 3, len(soc_points))
        dcir = np.clip(dcir, 30, 150)
        
        # 전압: SOC에 따라 선형 변화
        voltage = 3.0 + soc_points * 0.012 + np.random.normal(0, 0.05, len(soc_points))
        
        hppc_data['cycles'][cycle] = {
            'dcir': dcir,
            'voltage': voltage
        }
    
    return hppc_data

print("✅ generate_hppc_sample_data() 함수 정의 완료")

In [ ]:
def plot_hppc_analysis(hppc_data: Dict, hppc_settings: HPPCSettings):
    """
    HPPC/DCIR 분석 그래프
    1. SOC vs DCIR (여러 사이클)
    2. DCIR 변화 (사이클별)
    """
    
    fig, axes = plt.subplots(1, 2, figsize=(15, 5))
    fig.suptitle('HPPC/DCIR 분석 결과', fontsize=16, fontweight='bold')
    
    soc = hppc_data['soc']
    colors = ['#2E86AB', '#A23B72', '#F18F01', '#C73E1D', '#6A994E']
    
    # 1. SOC vs DCIR
    for i, (cycle, data) in enumerate(hppc_data['cycles'].items()):
        color = colors[i % len(colors)]
        axes[0].plot(soc, data['dcir'], 'o-', color=color, linewidth=2, markersize=6, label=f'Cycle {cycle}')
    
    axes[0].set_xlabel('SOC (%)', fontsize=12)
    axes[0].set_ylabel('DCIR (mΩ)', fontsize=12)
    axes[0].set_title('1. SOC vs DCIR', fontsize=12, fontweight='bold')
    axes[0].grid(True, alpha=0.3)
    axes[0].legend()
    axes[0].invert_xaxis()  # SOC 100% → 0%
    
    # 2. DCIR 평균값 변화 (사이클별)
    cycles = list(hppc_data['cycles'].keys())
    avg_dcir = [np.mean(data['dcir']) for data in hppc_data['cycles'].values()]
    
    axes[1].plot(cycles, avg_dcir, 'o-', color='#2E86AB', linewidth=2, markersize=8)
    axes[1].set_xlabel('Cycle', fontsize=12)
    axes[1].set_ylabel('평균 DCIR (mΩ)', fontsize=12)
    axes[1].set_title('2. 평균 DCIR 변화', fontsize=12, fontweight='bold')
    axes[1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    # 통계
    print("\n=== HPPC/DCIR 통계 ===\n")
    print(f"분석 사이클: {hppc_settings.start_cycle} ~ {hppc_settings.end_cycle}")
    print(f"SOC 측정점: {len(soc)}개")
    print(f"\nDCIR 통계:")
    all_dcir = np.concatenate([data['dcir'] for data in hppc_data['cycles'].values()])
    print(f"  최소: {np.min(all_dcir):.2f} mΩ")
    print(f"  최대: {np.max(all_dcir):.2f} mΩ")
    print(f"  평균: {np.mean(all_dcir):.2f} mΩ")
    print(f"  표준편차: {np.std(all_dcir):.2f} mΩ")

print("✅ plot_hppc_analysis() 함수 정의 완료")

In [ ]:
# HPPC 분석 테스트
print("=== HPPC/DCIR 분석 테스트 ===\n")

hppc_settings = HPPCSettings()
hppc_settings.start_cycle = 1
hppc_settings.end_cycle = 5

# 샘플 데이터 생성 (실제 데이터 로딩은 복잡하므로 샘플 사용)
hppc_data = generate_hppc_sample_data(capacity, hppc_settings.start_cycle, hppc_settings.end_cycle)

plot_hppc_analysis(hppc_data, hppc_settings)

---
## Section 10: 통합 분석 함수

In [ ]:
def indiv_cycle_analysis(folder_path: str, settings: CycleAnalysisSettings) -> Dict:
    """
    개별 폴더 사이클 분석 (완전한 워크플로우)
    
    Returns:
        result (dict): {'capacity', 'cycle_data', 'cycler_type', 'stats'}
    """
    print(f"\n{'='*60}")
    print(f"개별 사이클 분석 시작: {os.path.basename(folder_path)}")
    print(f"{'='*60}\n")
    
    # 1. 데이터 로딩
    capacity, cycle_df, cycler_type = load_cycle_data_real(
        folder_path, settings.capacity, settings.ini_crate, settings.chkir
    )
    
    print(f"✅ 데이터 로딩 완료")
    print(f"   충방전기: {cycler_type}")
    print(f"   용량: {capacity} mAh")
    print(f"   사이클 수: {len(cycle_df)}")
    
    # 2. 필터링 (cutoff)
    if settings.cutoff > 0:
        mask = cycle_df['Dchg'] >= settings.cutoff
        cycle_df = cycle_df[mask]
        print(f"   Cutoff 적용 후: {len(cycle_df)} 사이클")
    
    # 3. 통계 계산
    stats = {
        'total_cycles': len(cycle_df),
        'initial_capacity': cycle_df['Dchg'].iloc[0],
        'final_capacity': cycle_df['Dchg'].iloc[-1],
        'capacity_retention': (cycle_df['Dchg'].iloc[-1] / cycle_df['Dchg'].iloc[0]) * 100,
        'avg_efficiency': cycle_df['Eff'].mean(),
        'avg_dcir': cycle_df['dcir'].mean(),
        'avg_temp': cycle_df['Temp'].mean()
    }
    
    # 4. 그래프
    plot_cycle_analysis(cycle_df, capacity, settings)
    
    # 5. Excel 저장 (옵션)
    if settings.save_excel:
        excel_path = os.path.join(os.path.dirname(folder_path), 
                                  f"{os.path.basename(folder_path)}_cycle_analysis.xlsx")
        with pd.ExcelWriter(excel_path, engine='xlsxwriter') as writer:
            cycle_df.to_excel(writer, sheet_name='Cycle_Data')
            pd.DataFrame([stats]).to_excel(writer, sheet_name='Statistics', index=False)
        print(f"\n✅ Excel 저장: {excel_path}")
    
    return {
        'capacity': capacity,
        'cycle_data': cycle_df,
        'cycler_type': cycler_type,
        'stats': stats
    }

print("✅ indiv_cycle_analysis() 함수 정의 완료")

In [ ]:
def charge_discharge_profile_analysis(folder_path: str, settings: ProfileSettings, profile_type: str = 'charge') -> Dict:
    """
    충전/방전 프로파일 분석 (완전한 워크플로우)
    
    Returns:
        result (dict): {'profiles', 'cycle_nums', 'peaks'}
    """
    print(f"\n{'='*60}")
    title = '충전' if profile_type == 'charge' else '방전'
    print(f"{title} 프로파일 분석 시작: {os.path.basename(folder_path)}")
    print(f"{'='*60}\n")
    
    # 1. 용량 확인
    capacity = settings.capacity
    if capacity == 0:
        capacity = name_capacity(os.path.basename(folder_path))
        if capacity == 0:
            capacity = 4500
    
    print(f"용량: {capacity} mAh")
    
    # 2. 사이클 번호 파싱
    cycle_nums = [int(x) for x in settings.cycle_nums.split()]
    print(f"분석 사이클: {cycle_nums}")
    
    # 3. 프로파일 로딩
    profiles = []
    for cycle_num in cycle_nums:
        profile = load_profile_data_real(folder_path, capacity, cycle_num, profile_type)
        if profile:
            profiles.append(profile)
    
    print(f"\n✅ {len(profiles)}개 프로파일 로딩 완료")
    
    # 4. 그래프
    if profiles:
        plot_profile_analysis(profiles, cycle_nums[:len(profiles)], profile_type)
    
    # 5. Peak 분석 (마지막 사이클)
    peaks_info = []
    if profiles:
        last_profile = profiles[-1]
        peaks, properties = find_peaks(last_profile['dqdv'], prominence=settings.peak_prominence, distance=20)
        
        for peak_idx in peaks:
            peaks_info.append({
                'voltage': last_profile['voltage'][peak_idx],
                'dqdv': last_profile['dqdv'][peak_idx]
            })
    
    return {
        'profiles': profiles,
        'cycle_nums': cycle_nums[:len(profiles)],
        'peaks': peaks_info
    }

print("✅ charge_discharge_profile_analysis() 함수 정의 완료")

In [ ]:
def hppc_dcir_analysis(folder_path: str, settings: HPPCSettings) -> Dict:
    """
    HPPC/DCIR 분석 (완전한 워크플로우)
    
    Returns:
        result (dict): {'hppc_data', 'stats'}
    """
    print(f"\n{'='*60}")
    print(f"HPPC/DCIR 분석 시작: {os.path.basename(folder_path)}")
    print(f"{'='*60}\n")
    
    # 1. 용량 확인
    capacity = settings.capacity
    if capacity == 0:
        capacity = name_capacity(os.path.basename(folder_path))
        if capacity == 0:
            capacity = 4500
    
    print(f"용량: {capacity} mAh")
    print(f"분석 사이클: {settings.start_cycle} ~ {settings.end_cycle}")
    
    # 2. HPPC 데이터 로딩/생성
    # (실제 데이터 로딩은 복잡하므로 샘플 사용)
    hppc_data = generate_hppc_sample_data(capacity, settings.start_cycle, settings.end_cycle)
    
    print(f"\n✅ HPPC 데이터 로딩 완료")
    
    # 3. 그래프
    plot_hppc_analysis(hppc_data, settings)
    
    # 4. 통계
    all_dcir = np.concatenate([data['dcir'] for data in hppc_data['cycles'].values()])
    stats = {
        'min_dcir': np.min(all_dcir),
        'max_dcir': np.max(all_dcir),
        'avg_dcir': np.mean(all_dcir),
        'std_dcir': np.std(all_dcir)
    }
    
    return {
        'hppc_data': hppc_data,
        'stats': stats
    }

print("✅ hppc_dcir_analysis() 함수 정의 완료")

---
## Section 11: 종합 예제 실행

In [ ]:
# 예제 1: 개별 사이클 분석
print("\n" + "="*80)
print("예제 1: 개별 사이클 분석")
print("="*80)

settings = CycleAnalysisSettings()
settings.capacity = 0  # 자동 산정
settings.cycle_range_x = 0  # 자동 범위
settings.save_excel = False

if rawdata_folders:
    folder_path = rawdata_folders[0]['path']
else:
    folder_path = os.path.join(RAWDATA_PATH, "Sample_4500mAh_Test")

result = indiv_cycle_analysis(folder_path, settings)
print(f"\n✅ 분석 완료: {result['stats']['total_cycles']} 사이클")

In [ ]:
# 예제 2: 충전 프로파일 분석
print("\n" + "="*80)
print("예제 2: 충전 프로파일 분석")
print("="*80)

profile_settings = ProfileSettings()
profile_settings.cycle_nums = "2 10 50"
profile_settings.smooth_window = 51

result = charge_discharge_profile_analysis(folder_path, profile_settings, 'charge')
print(f"\n✅ 분석 완료: {len(result['profiles'])}개 프로파일, {len(result['peaks'])}개 Peak")

In [ ]:
# 예제 3: HPPC/DCIR 분석
print("\n" + "="*80)
print("예제 3: HPPC/DCIR 분석")
print("="*80)

hppc_settings = HPPCSettings()
hppc_settings.start_cycle = 1
hppc_settings.end_cycle = 5

result = hppc_dcir_analysis(folder_path, hppc_settings)
print(f"\n✅ 분석 완료: 평균 DCIR = {result['stats']['avg_dcir']:.2f} mΩ")

---
## Section 12: 사용 가이드 및 요약

### 📚 주요 함수 요약

#### 1. 데이터 로딩
- `load_cycle_data_real()`: 사이클 데이터 자동 로딩 (PNE/TOYO/샘플)
- `load_profile_data_real()`: 프로파일 데이터 자동 로딩

#### 2. 통합 분석
- `indiv_cycle_analysis()`: 개별 폴더 사이클 분석
- `charge_discharge_profile_analysis()`: 충방전 프로파일 분석
- `hppc_dcir_analysis()`: HPPC/DCIR 분석

#### 3. 그래프
- `plot_cycle_analysis()`: 6개 사이클 그래프
- `plot_profile_analysis()`: 프로파일 + dQdV
- `plot_hppc_analysis()`: SOC vs DCIR

### 💡 사용 팁

1. **자동 모드 전환**: 실제 데이터가 없으면 자동으로 샘플 데이터 사용
2. **용량 자동 산정**: `capacity=0`으로 설정하면 폴더명 또는 데이터에서 자동 추출
3. **Excel 저장**: `settings.save_excel = True`로 결과 저장
4. **그래프 커스터마이징**: Settings 클래스의 파라미터 조정

### 📖 참고 문서

- **NOTEBOOK_README.md**: 상세한 사용 설명서
- **UI_Button_Function_Flow.md**: 전체 기능 및 Flow 문서
- **BatteryDataTool.py**: 원본 소스 코드

### ✅ 체크리스트

- [x] 기본 유틸리티 함수 (9개)
- [x] 파일 시스템 함수 (3개)
- [x] 데이터 로딩 (자동 모드 전환)
- [x] 사이클 분석 그래프 (6개)
- [x] 프로파일 분석 (dQdV + Peak)
- [x] HPPC/DCIR 분석 (SOC 기반)
- [x] 통합 분석 함수
- [x] 샘플 데이터 Fallback

---

**버전**: 1.0  
**최종 수정**: 2025-10-28  
**기반**: BatteryDataTool.py, UI_Button_Function_Flow.md